**This ipynb file helps prepare some stuff for the masks.**

The functional connectomes have rows/columns representing the ROIs starting from ROI 1 (ROI 0 is background and ignored). They go in order.

In [2]:
# imports
import numpy as np
import csv
import nibabel as nib
import os
from contextlib import ExitStack

In [ ]:
# Function to calculate which 7-parcel ROIs (basically which functional network) each parcel of a non 7-parcel parcellation belongs in
# Outputs a csv file
# Deprecated

path_to_7_parcel = 'cambridge nii files/template_cambridge_basc_multiscale_sym_scale007.nii'

def label_rois_into_csv(path_to_template, output_dir='parcel_7-roi/'):
    nii_data_7 = nib.load(path_to_7_parcel).get_fdata()
    nii_data = nib.load(path_to_template).get_fdata()
    rois = np.unique(nii_data)[1:].astype(int) #skip the 0th parcel which is the background

    # Loop over parcels and check which 7-parcel it belongs to
    roi_mapping = {}

    for roi in rois:
        voxels = np.argwhere(nii_data == roi)
        voxelDict = {key: 0 for key in range(1,8)}
        for voxel in voxels:
            x, y, z = voxel
            roi_7 = nii_data_7[x, y, z]
            voxelDict[roi_7] += 1
        roi_mapping[roi] = voxelDict

    with open(os.path.join(output_dir, f'{len(rois)}-scale_voxel_distribution.csv'), mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header row
        writer.writerow(['ROI'] + list(roi_mapping[1].keys()))
        
        # Write data rows
        for key, values in roi_mapping.items():
            writer.writerow([key] + list(values.values()))
            
label_rois_into_csv('cambridge nii files/template_cambridge_basc_multiscale_sym_scale325.nii')

In [4]:
# Read MIST_PARCEL_ORDER.csv to determine parents and children of each ROI
# roi_parents holds the parent of each parent of a roi at any given scale, all the way to the 7-parcel network it belongs to
parcel_order_filepath = 'Parcel_Information/MIST_PARCEL_ORDER.csv'
scales = [7, 12, 20, 36, 64, 122, 197, 325, 444, 'atom']
roi_parents = {scale : {} for scale in scales}

with open(parcel_order_filepath, 'r') as file:
    reader = csv.reader(file, delimiter=',')
    next(reader, None)  # skip header
    for row in reader:
        for i, roi in enumerate(row):
            roi = int(roi)
            scale = scales[i]
            if roi not in roi_parents[scale]:
                parents = [int(roi) for roi in row[:i][::-1]]
                roi_parents[scale][roi] = parents

def get_roi_parents(roi, scale):
    return roi_parents[scale][roi]

In [11]:
# Function to make .node and .csv files with appropriate labeling for graphing in BrainNet Viewer and Gephi
# Get node locations as centroids of each parcel, save nodes to .node and csv file

def makeNodes(scale, nii_filename="", coloring='s', node_filename=''):
    if node_filename == '':
        node_filename=f'Parcel_Information/nodes_{scale}'

    assert coloring in ['s', '7']

    parcel_labels_file = f'Parcel_Information/MIST_{scale}.csv'
    roiDict = {}

    # read the MIST parcel labels csv file for the scale
    with open(parcel_labels_file, 'r') as file:
        reader = csv.DictReader(file, delimiter=';')
        for row in reader:
            row['parent'] = get_roi_parents(int(row['roi']), scale)
            roiDict[int(row['roi'])] = row

    # save node coords to .node file
    # each roi has its own module (therefore same color) for now
    with open(f'{node_filename}.node', mode='w', newline='') as nodeFile:
        for roi in roiDict.keys():
            row = roiDict[roi]
            # decide what to put in the color column
            if coloring == 's':
                color = row['roi']
            elif coloring == '7':
                color = row['parent'][-1]   #get the root parent of the roi (which is the 7-parcel network)
            
            nodeFile.write(f"{row['x']}\t{row['y']}\t{row['z']}\t{color}\t{row['size']}\t{row['label']}\n")

    # save nodes to csv file in gephi format
    with open(f'{node_filename}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ID'] + [key for key in roiDict[1].keys()] + ['7-parcel'])
        for roi in roiDict.keys():
            row = roiDict[roi]
            writer.writerow([row['roi']] + [row[key] for key in row.keys()] + [row['parent'][-1]]) 

    if nii_filename != "":
        # second .node file using old method of averaging parcels
        nii_img = nib.load(nii_filename)
        parcel_data = nii_img.get_fdata()

        # Find unique parcel indices (excluding zero if it represents the background)
        unique_parcels = np.unique(parcel_data)
        unique_parcels = unique_parcels[unique_parcels != 0]

        affine = nii_img.affine
        parcel_centroids = {}
        for parcel in unique_parcels:
            parcel_voxels = np.argwhere(parcel_data == parcel)
            world_coords = nib.affines.apply_affine(affine, parcel_voxels)
            
            # Compute the centroid (mean) of the coordinates for this parcel
            centroid = np.mean(world_coords, axis=0)
            parcel_centroids[int(parcel)] = centroid

        # save node coords to .node file
        with open(f'{node_filename}_avgs.node', mode='w', newline='') as f:
            for i in range(1,len(unique_parcels)+1, 1):
                for x in parcel_centroids[i]:
                    f.write(f'{x}\t')
                row = roiDict[i]
                f.write(f"{row['roi']}\t{row['size']}\t{row['label']}\n")    #write color number, size (set to 1 for now), label is parcel number

makeNodes(197, coloring='s', node_filename=f'Parcel_Information/nodes_{197}_7-coloring')